Selenium test

In [22]:
import os
import time
import wget
import json
import urllib
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options

In [12]:
VALID_EXTS = ['.png', '.jpg', '.gif', '.gifv']

In [13]:
def request(url, headers=None):
    if headers is None:
        headers = {
            'Content-Type': 'text/html',
            'Accept-Encoding': 'identity'
        }
    response = requests.get(url, headers=headers)
    return response

In [14]:
def page_down(page, n=2):
    for _ in range(n):
        page.send_keys(Keys.PAGE_DOWN)

In [15]:
baseurl = 'https://www.tumblr.com/search'

In [16]:
search = 'cowboy bebop'
url = urllib.parse.quote_plus(search)
url = f'{baseurl}/{url}'
url

'https://www.tumblr.com/search/cowboy+bebop'

In [17]:
options = Options()
options.add_argument('--headless')

In [18]:
driver = webdriver.Chrome(options=options)
driver.get(url)
page = driver.find_element_by_tag_name("html")

In [19]:
photos = []
for _ in range(5):
    html = driver.execute_script("return document.documentElement.innerHTML;")
    soup = BeautifulSoup(html, 'html.parser')

    articles = soup.find_all('article')
    for i, article in enumerate(articles, start=1):
        print(f'Processing {i}/{len(articles)}')
        data = json.loads(article.get('data-json'))
        photoset = data.get('photoset_photos', None)
        if photoset is None:
            post_url = data['share_popover_data']['post_url']
            response = request(post_url)
            soup = BeautifulSoup(response.content, 'html.parser')

            subarticles = soup.find_all('article')
            for subarticle in subarticles:
                images = subarticle.find_all('img')
                for image in images:
                    image = image.get('src')
                    ext = os.path.basename(image)[-1]
                    if ext in VALID_EXTS:
                        if image in photos:
                            print("Photo already captured. Continuing")
                            continue
                        photos.append(image)
        else:
            for photo in photoset:
                if photo['high_res'] in photos:
                    print("Photo already captured. Continuing")
                    continue
                photos.append(photo['high_res'])

    page_down(page)
    time.sleep(30)

Processing 1/20
Processing 2/20
Processing 3/20
Processing 4/20
Processing 5/20
Processing 6/20
Processing 7/20
Processing 8/20
Processing 9/20
Processing 10/20
Processing 11/20
Processing 12/20
Processing 13/20
Processing 14/20
Processing 15/20
Processing 16/20
Processing 17/20
Processing 18/20
Processing 19/20
Processing 20/20
Processing 1/20
Photo already captured. Continuing
Photo already captured. Continuing
Photo already captured. Continuing
Photo already captured. Continuing
Photo already captured. Continuing
Photo already captured. Continuing
Photo already captured. Continuing
Photo already captured. Continuing
Photo already captured. Continuing
Processing 2/20
Photo already captured. Continuing
Photo already captured. Continuing
Photo already captured. Continuing
Photo already captured. Continuing
Photo already captured. Continuing
Processing 3/20
Processing 4/20
Processing 5/20
Processing 6/20
Processing 7/20
Photo already captured. Continuing
Photo already captured. Continui

In [20]:
photos

['https://64.media.tumblr.com/a4bd5eb4f1a65dcabc02176ac69edc9d/tumblr_puys87ZJnE1tkezr8o7_r1_540.gifv',
 'https://64.media.tumblr.com/4daf8fd752b1808d72bdddd091415b7a/tumblr_puys87ZJnE1tkezr8o5_r1_540.gifv',
 'https://64.media.tumblr.com/b61bdb65314d3985b9fb579d52010e18/tumblr_puys87ZJnE1tkezr8o4_r1_540.gifv',
 'https://64.media.tumblr.com/b4b9404369f38a3835c5d9af6bcde94f/tumblr_puys87ZJnE1tkezr8o2_540.gifv',
 'https://64.media.tumblr.com/8a0533f604288d0efc9222d68cc06faa/tumblr_puys87ZJnE1tkezr8o6_r1_540.gifv',
 'https://64.media.tumblr.com/0f59c56cca8c5e72c8119ea4042cb120/tumblr_puys87ZJnE1tkezr8o8_r1_540.gifv',
 'https://64.media.tumblr.com/0e09950eb20279c3459a48edbf691f6e/tumblr_puys87ZJnE1tkezr8o9_r1_540.gifv',
 'https://64.media.tumblr.com/79b6f420bfd73dc8c42628e1e41623df/tumblr_puys87ZJnE1tkezr8o1_540.gifv',
 'https://64.media.tumblr.com/3a20328660e0de7dcbd6b572c5e22546/tumblr_puys87ZJnE1tkezr8o3_540.gifv',
 'https://64.media.tumblr.com/2d5a714d6f2ce01720c043351cadbcc2/tumblr_ptf

In [23]:
base = 'images'
os.makedirs(base, exist_ok=True)
for photo in photos:
    wget.download(photo, out=base)